# Quickstart

A tour of this package's functionality! Buckle up because it's gonna be a fun one.

## Tools and Toolkits

### Tools: `@function_tool`
You can make any function into a tool usable by the model by decorating it with `@function_tool`, but you must do the following:
- The parameters must be type annotated
- The return type *should* be str, but not required (currently just stringified)
- It must be documented with a `'''docstring'''`, including each parameter (most [formats supported](https://github.com/rr-/docstring_parser), e.g. [Google-style](https://gist.github.com/redlotus/3bc387c2591e3e908c9b63b97b11d24e#file-docstrings-py-L67), [NumPy-style](https://gist.github.com/eikonomega/910512d92769b0cc382a09ae4de41771), sphinx-style, etc, see [this overview](https://gist.github.com/nipunsadvilkar/fec9d2a40f9c83ea7fd97be59261c400))

You can use the tool from python as you normally would, but also the annotated tool will be auto-parsed and its JSON schema and argument validator (using pydantic) will be available as attributes on the function:
- `tool.schema` is the (singular list of) JSON schema, which you can directly pass to the OpenAI API.
- `tool.lookup` is a dict lookup table of the function name to the (auto-validated) function call, meaning you can just receive the model-generated function call and pass it directly to the tool.

Other attributes (probably commonly used):
- `tool.validator` is the pydantic validator, callable with kwargs to validate the arguments. This is what is saved in the lookup table.
- `tool.tool_enabled` is a bool switch to enable/disable the tool. This is useful esp. for Toolkits, where you can dynamically enable/disable tools based on the context. Note that this only works if you wrap the tool in `ToolList` class or is part of a `Toolkit`.
- `tool.name` should ideally not be set directly, but passed as a kwarg to `@function_tool`. It is the name of the tool, as known to the model. If not set, it will be the name of the function.

In [4]:
from gpt_wrapper.tools import ToolList, Toolkit, function_tool, fail_with_message
import logging
logger = logging.getLogger(__name__)

In [5]:
@function_tool
def print_to_console(text: str) -> str:
    '''
    Print text to console

    Args:
        text: text to print
    '''
    print(text)
    return 'success' # ideally, we always return something to tell the model

# normal call
print_to_console('Hello from python!')

# call from lookup table
lookup = print_to_console.lookup
# this would be generated by GPT
name, arguments = 'print_to_console', {'text': 'Hello from GPT!'}
func = lookup[name]
func(arguments) # notice the lack of unpacking, we pass the args dict directly

Hello from python!
Hello from GPT!


'success'

In [6]:
from types import SimpleNamespace
import json
from gpt_wrapper.tools import call_requested_function

call = SimpleNamespace(name='print_to_console', arguments=json.dumps({'text': 'Hello from GPT!'}))
await call_requested_function(call, print_to_console.lookup)

awaiting thread
Hello from GPT!


'success'

In [7]:
# if the model generates an invalid argument, it is auto-validated
print(func({'text': 123}))
print(func({'content': "Whats up?"}))

# notice that it's returned as string, because it should be passed to the model to correct itself

Invalid Argument: 1 validation error for print_to_console
text
  Input should be a valid string [type=string_type, input_value=123, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/string_type
Invalid Argument: 1 validation error for print_to_console
text
  Field required [type=missing, input_value={'content': 'Whats up?'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.5/v/missing


Showing off some more goodies:
- Even async functions should seamlessly work, just don't forget to `await` them.
- `@fail_with_message(err)` is a decorator that will catch any exceptions thrown by the function and instead return the error message. This is useful for when you want to handle errors in a more graceful way than just crashing the model. It also takes an optional logger, which by default takes the `print` function, but any callable that takes a string will work, such as `logger.error` from the `logging` module.
- Usually, the `@function_tool` decorator will throw an assertion error if you forget to provide the description for any of the function or their parameters. If you really don't want to provide descriptions for some (or all), maybe because it's so self-explanatory or you need to save tokens, then you can explicitly turn off the docstring parsing by passing `@function_tool(check_description=False)`. This is not recommended, but it's there if you need it.

In [8]:
import asyncio

@function_tool(name="Fibonacci")
@fail_with_message("Error", logger=logger.error)
async def fib(n: int):
    '''
    Calculate the nth Fibonacci number

    Args:
        n: The index of the Fibonacci number to calculate
    '''
    if n < 0:
         raise ValueError("n must be >= 0")
    if n < 2:
        return n
    await asyncio.sleep(0.1)
    # return await fib(n-1) + await fib(n-2)
    # parallel
    return sum(await asyncio.gather(fib(n-1), fib(n-2)))

print(await fib(10), flush=True)
print(await fib.lookup['Fibonacci']({'n': -10}))
fib.schema

55


Tool call fib(n=-10) failed: n must be >= 0


Error: n must be >= 0


[{'type': 'function',
  'function': {'name': 'Fibonacci',
   'description': 'Calculate the nth Fibonacci number',
   'parameters': {'type': 'object',
    'properties': {'n': {'description': 'The index of the Fibonacci number to calculate',
      'type': 'integer'}},
    'required': ['n']}}}]

### Toolkits: `class Toolkit`
Toolkits are a collection of related function tools, esp. useful when they share a state. Also good for keeping the state bound to a single instance of the toolkit, rather than a global state.
To create a toolkit, simply subclass `Toolkit` and decorate its methods with `@function_tool`.
Enabled tools will be visible in the attributes:
- `toolkit.schema` is the JSON schema, which you can directly pass to the OpenAI API.
- `toolkit.lookup` is a dict lookup table of the function name to the (auto-validated) function call, meaning you can just receive the model-generated function call and pass it directly to the tool.

As you noticed, the interface for OpenAI API is the same for both tools and toolkits, so you can use them interchangeably, and to use multiple tools and toolkits, simply merge their schemas and lookups.

In [9]:
class Notepad(Toolkit):
    def __init__(self):
        super().__init__()
        self.content = "<Fill me in>"
    
    @function_tool
    def write(self, text: str):
        '''
        Write text to the notepad

        Args:
            text: The text to write
        '''
        self.content = text
    
    @function_tool(check_description=False)
    def read(self):
        return self.content
    
notes = Notepad()
notes.write("Hello, world!")
print(notes.read())

Hello, world!


In [10]:
notes.lookup['write']({'text': "Shhh... here's a secret: 42"})
notes.lookup['read']({})

"Shhh... here's a secret: 42"

## ChatGPT model and MessageHistory

### ChatGPT: `class ChatGPT`

Simple wrapper that keeps a message history object and optional toolkit (TODO: make into list of toolkits).
The object is callable and simply takes a user prompt string and returns the model response string.

### MessageHistory: `class MessageHistory`

A simple class that keeps track of the message history, including the user and assistnat messages, along with system and tool call results. Special subclasses could be defined for things like:
- Rolling history (e.g. last 10 messages)
- Dynamic history based on context (e.g. vector embedding)
- System message that constantly updates (e.g. time, weather, etc.)
- Few-shot learning, i.e. pre-populating the history with some demonstration messages

In [12]:
from gpt_wrapper.models import ChatGPT
from gpt_wrapper.messages import MessageHistory, msg

gpt = ChatGPT(
    model = 'gpt-3.5-turbo',
    tools = ToolList(notes, print_to_console, fib),
    messages = MessageHistory()
)

In [10]:
response = await gpt("What's on my notepad?")

[User]: What's on my notepad?
[Tool Calls]: [ChatCompletionMessageToolCall(id='call_5ZEDbC9xRELxX62MQgrt6DIb', function=Function(arguments='{}', name='read'), type='function')]
awaiting thread
[Tool Result]: Shhh... here's a secret: 42
[ChatGPT]: On your notepad, you have the number 42 written as a secret.
[Final Response]: On your notepad, you have the number 42 written as a secret.
On your notepad, you have the number 42 written as a secret.


In [11]:
response = await gpt("Can you calculate the 8th fibonacci number, add it to the number in my notes, and write it? also print it to console as well.")

[User]: Can you calculate the 8th fibonacci number, add it to the number in my notes, and write it? also print it to console as well.
[Tool Calls]: [ChatCompletionMessageToolCall(id='call_5ZEDbC9xRELxX62MQgrt6DIb', function=Function(arguments='{\n  "n": 8\n}', name='Fibonacci'), type='function')]
awaiting coroutine
[Tool Result]: 21
[Tool Calls]: [ChatCompletionMessageToolCall(id='call_Whczy5fQYmrgelRtDYzGZ5Xx', function=Function(arguments='{\n  "text": "63"\n}', name='write'), type='function')]
awaiting thread
[Tool Result]: None
[Tool Calls]: [ChatCompletionMessageToolCall(id='call_AaivIquM9AfuSW1dMi3GXyQf', function=Function(arguments='{\n  "text": "63"\n}', name='print_to_console'), type='function')]
awaiting thread
63
[Tool Result]: success
[ChatGPT]: I have calculated the 8th Fibonacci number, which is 21. I have added it to the number on your notepad, which is 42, and the result is 63. I have written this number on your notepad and printed it to the console as well.
[Final Respo

In [12]:
notes.read()

'63'

In [13]:
gpt.messages.history

[{'role': 'system', 'content': 'You are a helpful assistant'},
 {'role': 'user', 'content': "What's on my notepad?"},
 {'role': 'assistant',
  'tool_calls': [{'id': 'call_5ZEDbC9xRELxX62MQgrt6DIb',
    'function': {'arguments': '{}', 'name': 'read'},
    'type': 'function'}]},
 {'role': 'tool',
  'content': "Shhh... here's a secret: 42",
  'tool_call_id': 'call_5ZEDbC9xRELxX62MQgrt6DIb'},
 {'content': 'On your notepad, you have the number 42 written as a secret.',
  'role': 'assistant'},
 {'role': 'user',
  'content': 'Can you calculate the 8th fibonacci number, add it to the number in my notes, and write it? also print it to console as well.'},
 {'role': 'assistant',
  'tool_calls': [{'id': 'call_5ZEDbC9xRELxX62MQgrt6DIb',
    'function': {'arguments': '{\n  "n": 8\n}', 'name': 'Fibonacci'},
    'type': 'function'}]},
 {'role': 'tool',
  'content': '21',
  'tool_call_id': 'call_5ZEDbC9xRELxX62MQgrt6DIb'},
 {'role': 'assistant',
  'tool_calls': [{'id': 'call_Whczy5fQYmrgelRtDYzGZ5Xx',


### Customizing our ChatGPT Class

Our `Assistant` classes are based on the concept of Event Generators, i.e. a python iterator that yields different events. The above `__call__()` method is an example of an Event Handler that simply calls the event generators and reacts to the emitted events it cares about and ignores the rest. This is a very flexible design pattern that allows us to easily customize the behavior of the model.

Let's override the `__call__()` method to handle some more events we care about:
- let's print the streamed partial responses directly
- let's use the jupyter `display` function to display the full response in a nice Markdown format

Basically like the following simple toy example:

In [13]:
import time
from IPython.display import display, Markdown

def gen_response(txt):
    for i in range(len(txt)):
        yield txt[:i+1]
        time.sleep(0.01)

t = """
Hello, world!

# How are you?
I love earth! And everything on it!

```python
print("Hello, world!")
def fib(n):
    if n < 0:
         raise ValueError("n must be >= 0")
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)
```
Here's a list of things I like:
- Earth
- The Moon
- Mars
- You
- Myself
- GPT-Wrapper

Anyways, **goodbye, have a nice day**!
"""

for r in gen_response(t):
    display(Markdown(r), clear=True)


Hello, world!

# How are you?
I love earth! And everything on it!

```python
print("Hello, world!")
def fib(n):
    if n < 0:
         raise ValueError("n must be >= 0")
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)
```
Here's a list of things I like:
- Earth
- The Moon
- Mars
- You
- Myself
- GPT-Wrapper

Anyways, **goodbye, have a nice day**!


In [19]:
from gpt_wrapper.models import ChatGPT

class NotebookGPT(ChatGPT):
    async def __call__(self, prompt: str, **kwargs):

        output = '' # accumulate output here

        async for event in self.response_events(prompt, **kwargs):
            match event:
                case self.ResponseStartEvent():
                    output += f'**[USER]:** {event.prompt}\n\n**[GPT]:** '
                    display(Markdown(output), clear=True)
                
                case self.PartialCompletionEvent():
                    try:
                        if d:=event.chunk.choices[0].delta.content:
                            output += d
                            display(Markdown(output), clear=True)
                    except:
                        pass

gpt = NotebookGPT(MessageHistory([]))

In [20]:
await gpt("Hey! How are you?")

**[USER]:** Hey! How are you?

**[GPT]:** Hello! I am an AI and do not possess emotions, but I am here to assist you. How can I help you today?

In [21]:
await gpt("Can you give me a python implementation of the fibonacci sequence iteratively?")

**[USER]:** Can you give me a python implementation of the fibonacci sequence iteratively?

**[GPT]:** Of course! Here's a Python implementation of the Fibonacci sequence using an iterative approach:

```python
def fibonacci_iterative(n):
    if n <= 0:
        return []
    elif n == 1:
        return [0]
    elif n == 2:
        return [0, 1]
    
    # Start with the first two numbers of the sequence
    fib_seq = [0, 1]

    # Generate the Fibonacci sequence up to the desired length
    while len(fib_seq) < n:
        next_num = fib_seq[-1] + fib_seq[-2]
        fib_seq.append(next_num)
    
    return fib_seq
```

In this implementation, we check for cases where the sequence length is less than or equal to 0, 1, or 2 to handle special cases. Then, we start with the first two numbers of the sequence (0 and 1) and keep generating the next numbers by summing up the last two numbers. The process continues until we have the desired length of the Fibonacci sequence.

You can call the function `fibonacci_iterative(n)` to get the Fibonacci sequence of length `n`. For example, `fibonacci_iterative(10)` would return `[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]`.

### Even more customization

So far we've only overriden the Event Handlers, with which you can go pretty far. However, you can also override the Event Generators to customize the behavior even more. You could define new Events and then yield them in your custom overriden Event Generators.

This could be useful when you're looking into advanced use cases, such as Code Interpreter with a "hacked" python tool, in order to get around the issue of all tool arguments being in JSON format, and the model often makes syntax mistakes in writing long blocks of code while simultaneously encoding it in a JSON string format (escaping new lines and quotes, etc.). For this, you would override the tool text message content handler (currently a TODO).

Another example could be for human-in-the-loop tools, e.g. you want the user to explictly confirm the tool call before it's actually executed for critical tools. For this you could yield a new Event called `ConfirmToolCall` and then override the `tool_events` generator to yield this event when needed.